In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
# Image size and batch size
img_size = (128, 128)
batch_size = 32

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

2025-08-02 18:14:42.922109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754138682.940933    6288 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754138682.946739    6288 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754138682.961543    6288 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754138682.961565    6288 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754138682.961566    6288 computation_placer.cc:177] computation placer alr

In [2]:
train_gen = train_datagen.flow_from_directory(
    "Dataset_Split/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Validation set
val_gen = val_datagen.flow_from_directory(
    "Dataset_Split/val",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Test set
test_gen = test_datagen.flow_from_directory(
    "Dataset_Split/test",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False  # Important for consistent evaluation
)

Found 3500 images belonging to 2 classes.
Found 750 images belonging to 2 classes.
Found 750 images belonging to 2 classes.


In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

base_model = ResNet50(
    input_shape = (128,128,3),
    include_top = False,
    weights='imagenet'
)

base_model.trainable = False

model = Sequential(
    [
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

I0000 00:00:1754138689.855414    6288 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4057 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5


In [ ]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
import PIL

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)
I0000 00:00:1754138704.741318    6404 service.cc:152] XLA service 0x7feb8004d690 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754138704.741367    6404 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2025-08-02 18:15:04.961151: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1754138706.899734    6404 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-02 18:15:07.836914: I external/local_xla/xla/service/gpu/autotuning/conv_algor

  1/110 ━━━━━━━━━━━━━━━━━━━━ 22:46 13s/step - accuracy: 0.5938 - loss: 0.0000e+00

I0000 00:00:1754138710.570172    6404 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


107/110 ━━━━━━━━━━━━━━━━━━━━ 1s 451ms/step - accuracy: 0.4059 - loss: 0.0000e+00

2025-08-02 18:16:00.870498: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.162 = (f32[12,64,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[12,64,32,32]{3,2,1,0} %bitcast.5132, f32[64,64,3,3]{3,2,1,0} %bitcast.5139, f32[64]{0} %bitcast.5141), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/resnet50_1/conv2_block1_2_conv_1/convolution" source_file="/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-08-02 18:16:01.108608: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 486ms/step - accuracy: 0.4058 - loss: 0.0000e+00

2025-08-02 18:16:19.064815: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.162 = (f32[14,64,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[14,64,32,32]{3,2,1,0} %bitcast.4657, f32[64,64,3,3]{3,2,1,0} %bitcast.4664, f32[64]{0} %bitcast.4666), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/resnet50_1/conv2_block1_2_conv_1/convolution" source_file="/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-08-02 18:16:19.291150: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549

110/110 ━━━━━━━━━━━━━━━━━━━━ 83s 650ms/step - accuracy: 0.4017 - loss: 0.0000e+00 - val_accuracy: 0.4000 - val_loss: 0.0000e+00
Epoch 2/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 24s 222ms/step - accuracy: 0.4000 - loss: 0.0000e+00 - val_accuracy: 0.4000 - val_loss: 0.0000e+00
Epoch 3/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 23s 212ms/step - accuracy: 0.4000 - loss: 0.0000e+00 - val_accuracy: 0.4000 - val_loss: 0.0000e+00
Epoch 4/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 23s 206ms/step - accuracy: 0.4000 - loss: 0.0000e+00 - val_accuracy: 0.4000 - val_loss: 0.0000e+00
Epoch 5/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 23s 206ms/step - accuracy: 0.4000 - loss: 0.0000e+00 - val_accuracy: 0.4000 - val_loss: 0.0000e+00
Epoch 6/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 23s 206ms/step - accuracy: 0.4000 - loss: 0.0000e+00 - val_accuracy: 0.4000 - val_loss: 0.0000e+00
Epoch 7/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 23s 205ms/step - accuracy: 0.4000 - loss: 0.0000e+00 - val_accuracy: 0.4000 - val_loss: 0.0000e+00
Epoch 8/10
110/110 ━━━━━━━━━━━━━━━━━━━